## To create a vector DB with embeddings of the codebase


Steps:

- We will upload all python project files using the langchain_community.document_loaders.TextLoader
- iterate over the files in the LangChain repository and loads every .py file
- splitting stragey :
    - Keep top-level functions and classes together (into a single document)
    - Put remaining code into a separate document
    - Retains metadata about where each split comes from  

Required libraries:

langchain-openai tiktoken langchain-chroma langchain GitPython

Reference: https://python.langchain.com/v0.1/docs/use_cases/code_understanding/

In [1]:
# ! pip install --upgrade --quiet tiktoken langchain-chroma 
# ! pip install langchain-community

In [2]:
from dotenv import load_dotenv
import os
load_dotenv('/Users/sjr11/Work/SelfLearning/Git Chat Agent/ai_git_assistant/.env')
print(os.getenv('LANGCHAIN_PROJECT'))

gitchat_assistant


In [3]:
from git import Repo
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language

In [4]:
# Clone Repo locally if not already done 
repo_path = '/Users/sjr11/Work/SelfLearning/sql_python_etl/python_sql_etl'
repo_url = "https://github.com/shailu1309/python_sql_etl.git"
if not os.path.exists(repo_path):
    # Clone the repository if it doesn't exist
    repo = Repo.clone_from(repo_url, to_path=repo_path)
else:
    # Open the existing repository
    repo = Repo(repo_path)
    print("Repository already exists at the specified path.")

Repository already exists at the specified path.


In [5]:
# Load
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".py"],
    exclude=["**/non-utf8-encoding.py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500),
)
documents = loader.load()
len(documents)

4

In [12]:
documents[0]

Document(metadata={'source': '/Users/sjr11/Work/SelfLearning/sql_python_etl/python_sql_etl/create_mock_db.py', 'language': <Language.PYTHON: 'python'>}, page_content="import sqlite3\n\nconnection = sqlite3.connect('sqlite_mock.db')\ncursor = connection.cursor()\n\ncursor.execute('''CREATE TABLE IF NOT EXISTS table1 \n               (id INTEGER, col1 TEXT)''')\ncursor.execute('''CREATE TABLE IF NOT EXISTS table2 \n               (id INTEGER, col2 TEXT)''')\n\ncursor.executemany('INSERT INTO table1 (id, col1) VALUES (?, ?)', [\n            (1, 'dummy'), (2, 'example'), (3, ''), (4, 'test')])\ncursor.executemany('INSERT INTO table2 (id, col2) VALUES (?, ?)', [\n            (1, 'value1'), (2, None), (3, 'value3'), (4, 'value4')])\n\n# Commit the changes to the database\nconnection.commit()\n\n# Query the database to retrieve data\ncursor.execute('SELECT * FROM table1;')\nrows = cursor.fetchall()\nprint(len(rows))\nconnection\nconnection.close()\n\n")

In [13]:
# Splitting - Split the Document into chunks for embedding and vector storage.
# We can use RecursiveCharacterTextSplitter w/ language specified.
from langchain_text_splitters import RecursiveCharacterTextSplitter

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)
texts = python_splitter.split_documents(documents)
len(texts)

5

### creating embedding with Retrieval QA
We need to store the documents in a way we can semantically search for their content.

The most common approach is to embed the contents of each document then store the embedding and document in a vector store.

When setting up the vectorstore retriever:

We test max marginal relevance for retrieval
And 8 documents returned


In [14]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))
retriever = db.as_retriever(
    search_type="mmr",  # Also test "similarity"
    search_kwargs={"k": 8},
)

In [15]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4")

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
        (
            "user",
            "Given the above conversation, generate a search query to look up to get information relevant to the conversation",
        ),
    ]
)

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)
document_chain = create_stuff_documents_chain(llm, prompt)

qa = create_retrieval_chain(retriever_chain, document_chain)

In [16]:
question = "What is the project about?"
result = qa.invoke({"input": question})
result["answer"]

Number of requested results 20 is greater than number of elements in index 5, updating n_results = 5


"The project appears to be about data extraction from multiple tables in a database. The script written in Python uses logging to monitor the data extraction process from three tables - table1, table2, and table3. \n\nThe extraction functions use SQLAlchemy to connect to a PostgreSQL database and execute queries defined in the functions `inefficient_query_1()`, `inefficient_query_2()`, and `inefficient_query_3()`. Each query returns a pandas DataFrame. \n\nAdditionally, an SQLite database is created and populated with data for tables 'table1' and 'table2'. \n\nThere's also a testing component where a pytest function is defined to compare the results of two SQL queries. \n\nIt appears the project might be about optimizing these 'inefficient' queries, as it includes a testing setup to compare the results of modifications to these queries."

In [17]:
questions = [
    "What is the project about??",
    "Are there sql queries in the project?",
    "Can you identify the file names where sql queries are written in the project?"
]

for question in questions:
    result = qa.invoke({"input": question})
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

Number of requested results 20 is greater than number of elements in index 5, updating n_results = 5


-> **Question**: What is the project about?? 

**Answer**: The project is about extracting data from different tables in a database using Python. It includes creating and interacting with SQLite and PostgreSQL databases, executing SQL queries, and handling the data using pandas DataFrames. 

The main.py file contains the main function which logs the data extraction process and extracts data from three tables (table1, table2, table3) using the functions from the extract.py file. In case an error occurs during the extraction, it's logged and the program continues to the next table.

The extract.py file contains the SQL queries for data extraction and a function to execute the SQL queries and return the data as a pandas DataFrame.

There's also a test module that uses pytest to compare the results of two SQL queries to ensure they return the same results. This is done by parameterizing the test function with different pairs of SQL queries. 

The project also includes creating tables and i

Number of requested results 20 is greater than number of elements in index 5, updating n_results = 5


-> **Question**: Are there sql queries in the project? 

**Answer**: Yes, there are SQL queries in the project. These queries are used to interact with the databases 'sqlite_mock.db' and 'dummy_postgres'. For example, queries are used to create tables, insert data into tables, and retrieve data from tables. There are also inefficient queries defined in the 'extract.py' file which are used to extract data from the tables in 'dummy_postgres'. 



Number of requested results 20 is greater than number of elements in index 5, updating n_results = 5


-> **Question**: Can you identify the file names where sql queries are written in the project? 

**Answer**: The SQL queries are written in two files: 

1. In the initial part of the context, there are SQL queries in an unnamed file, which could be a test file based on the usage of pytest.
  
2. The second file is "extract.py" where inefficient queries for table1, table2, and table3 are defined. 

